In [1]:
import os
os.chdir('/Users/natepruitt/Projects/robocritic/')

In [2]:
from helper_functions.db_connection import connection
from helper_functions.query_functions import execute_read_query

Connection to MySQL DB successful


In [3]:
query = "SELECT * FROM game;"

In [4]:
print(execute_read_query(connection, query))

[(1, 'The Last Faith', datetime.date(2023, 11, 15), None)]


In [5]:
games = execute_read_query(connection, query)
last_faith = games[0]

In [6]:
last_faith_name = last_faith[1].replace(" ", "-").casefold()

In [7]:
publishers_query = "SELECT * FROM publisher"
publishers = execute_read_query(connection, publishers_query)
publishers

[(1, 'IGN', 'https://www.ign.com/articles/{game_name}-review', None)]

In [8]:
publishers[0][2].format(game_name = last_faith_name)

'https://www.ign.com/articles/the-last-faith-review'

In [9]:
from bs4 import BeautifulSoup
import requests
# soup = BeautifulSoup(, 'html.parser')

In [10]:
headermap = {"User-Agent": "Mac Firefox"}
ign = requests.get('https://www.ign.com/articles/the-last-faith-review', headers=headermap)

In [11]:
soup = BeautifulSoup(ign.content, 'html.parser')

In [12]:
articles = soup.find_all("p")

In [13]:
''.join(list(map(lambda x: x.get_text(), articles)))

"The Last Faith feels like a good cover of a song you love, or a solid genre movie that isn’t going to push any boundaries, but will scratch an itch for digestible, reliable entertainment. If you like 2D platforming through big, non-linear maps that open up as you unlock new ways to maneuver around their obstacles, grinding through chaff that can be surprisingly resilient and clever on your way to big challenging boss fights, and a dark world fresh out of an apocalyptic event that needs your help to save whatever’s left of it, then this game is for you. It plays all the hits respectably, and even adds a few of its own flourishes, like an armory of clever weapons and flashy spells. But it does not over achieve, and is often a reminder of how much more special the games it’s clearly inspired by are in comparison.I'll be honest with you: by the end of my 15-hour journey through The Last Faith’s snowy cities, fetid crypts, and dilapidated manors, and seeing two of assumptively many possibl

In [14]:
from openai import OpenAI

In [16]:
client = OpenAI()

In [30]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format= {"type": "json_object"},
  messages=[
    {"role": "system", "content": "You are a helpful assistant, designed to analyze video game reviews and return pros and cons."},
    {"role": "system", "content": "You will output your response in JSON. The response output will be a JSON with fields 'pros' and 'cons'"},
    {"role": "system", "content": "The JSON response output will have fields 'pros' and 'cons'. The values will be comma seperated lists of the pros and cons"},
    {"role": "system", "content": "The list of pros and cons should be generated ONLY from the content passed by the user. Do not use additional resources"},
    {"role": "user", "content": ''.join(list(map(lambda x: x.get_text(), articles)))}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='{\n  "pros": "Solid platforming gameplay, Clever weapons and spells, Satisfyingly macabre mood, Challenging bosses, Clear progression path, Fun selection of weapons",\n  "cons": "Lackluster story, Lack of memorable environments, Wild fluctuations in pacing, Lack of memorable boss designs, Limited options for character builds, Spells feel underutilized, Firearms feel unnecessary"\n}', role='assistant', function_call=None, tool_calls=None)


In [31]:
completion.choices[0].message.content

'{\n  "pros": "Solid platforming gameplay, Clever weapons and spells, Satisfyingly macabre mood, Challenging bosses, Clear progression path, Fun selection of weapons",\n  "cons": "Lackluster story, Lack of memorable environments, Wild fluctuations in pacing, Lack of memorable boss designs, Limited options for character builds, Spells feel underutilized, Firearms feel unnecessary"\n}'

In [32]:
import json

In [33]:
json.loads(completion.choices[0].message.content)

{'pros': 'Solid platforming gameplay, Clever weapons and spells, Satisfyingly macabre mood, Challenging bosses, Clear progression path, Fun selection of weapons',
 'cons': 'Lackluster story, Lack of memorable environments, Wild fluctuations in pacing, Lack of memorable boss designs, Limited options for character builds, Spells feel underutilized, Firearms feel unnecessary'}

In [35]:
completion.usage

CompletionUsage(completion_tokens=85, prompt_tokens=1865, total_tokens=1950)